In [30]:
import pandas as pd
import pytz

In [31]:
tempo = pd.read_csv('../Bases/Raw/INMET_SE_SP_A701_SAO PAULO - MIRANTE_01-01-2019_A_31-12-2019.CSV', sep=';', header=8, encoding='utf-8')
tempo

,Data,Hora UTC,"PRECIPITACAO TOTAL, HORARIO (mm)","PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)",PRESSAO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB),PRESSAO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB),RADIACAO GLOBAL (KJ/m²),"TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",TEMPERATURA DO PONTO DE ORVALHO (°C),TEMPERATURA MAXIMA NA HORA ANT. (AUT) (°C),TEMPERATURA MINIMA NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C),UMIDADE REL. MAX. NA HORA ANT. (AUT) (%),UMIDADE REL. MIN. NA HORA ANT. (AUT) (%),"UMIDADE RELATIVA DO AR, HORARIA (%)","VENTO, DIRECAO HORARIA (gr) ((gr))","VENTO, RAJADA MAXIMA (m/s)","VENTO, VELOCIDADE HORARIA (m/s)",Unnamed: 19
0,2019/01/01,0000 UTC,0,"925,6","925,6","925,2",NaN,"22,8","16,9",23,"22,7","17,3","16,8",71,68,69,123,7,"2,6",NaN
1,2019/01/01,0100 UTC,0,926,926,"925,6",NaN,"22,3","16,8","22,8","22,3","16,9","16,4",71,68,71,107,"6,8","2,5",NaN
2,2019/01/01,0200 UTC,0,"925,5",926,"925,5",NaN,"22,1",17,"22,3","22,1","17,1","16,8",73,71,73,109,"6,5","2,1",NaN
3,2019/01/01,0300 UTC,0,"924,7","925,5","924,7",NaN,"21,6","17,1","22,1","21,6","17,2",17,76,73,76,101,"5,8","1,6",NaN
4,2019/01/01,0400 UTC,0,"924,4","924,7","924,4",NaN,"21,6","17,1","21,6","21,5","17,2","17,1",76,75,76,107,"5,1","2,2",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,2019/12/31,1900 UTC,",2","920,4","920,7","920,1","575,9","26,2","15,9","27,8","25,2","18,8","15,5",63,52,53,74,"9,9",",6",NaN
8756,2019/12/31,2000 UTC,0,"920,4","920,7","920,3","600,3","24,9","18,1","26,6","24,8","18,4","15,7",66,52,66,37,"4,7","1,4",NaN
8757,2019/12/31,2100 UTC,0,920,"920,4","919,9","922,5","26,5","18,5","26,5","24,9","18,6","17,6",67,59,62,137,"4,3","1,5",NaN
8758,2019/12/31,2200 UTC,0,"920,5","920,5",920,"83,9",25,"18,7","26,5",25,"18,8","18,2",68,61,68,164,"4,1","1,7",NaN


In [32]:
tempo = tempo[['Data','Hora UTC','PRECIPITACAO TOTAL, HORARIO (mm)','TEMPERATURA MAXIMA NA HORA ANT. (AUT) (°C)','TEMPERATURA MINIMA NA HORA ANT. (AUT) (°C)','UMIDADE RELATIVA DO AR, HORARIA (%)','VENTO, VELOCIDADE HORARIA (m/s)']]
tempo = tempo.loc[(tempo['Data'] >= '2019/06/01') & (tempo['Data'] <= '2019/10/10')]

In [33]:

tempo['Datetime'] = pd.to_datetime(tempo['Data']+' '+tempo['Hora UTC'])

In [34]:
tempo['Datetime'] = tempo['Datetime'].dt.tz_convert(pytz.timezone('America/Sao_Paulo'))
tempo['Datetime'] = tempo['Datetime'].dt.tz_localize(None)

In [35]:
tempo["Temperatura Media"] = (tempo["TEMPERATURA MAXIMA NA HORA ANT. (AUT) (°C)"].str.replace(",", ".").astype(float) + tempo["TEMPERATURA MINIMA NA HORA ANT. (AUT) (°C)"].str.replace(",", ".").astype(float))/2

In [36]:
tempo.drop(["Data", "Hora UTC", "TEMPERATURA MAXIMA NA HORA ANT. (AUT) (°C)", "TEMPERATURA MINIMA NA HORA ANT. (AUT) (°C)"], axis=1, inplace=True)

In [37]:
tempo.columns = ["Precipitacao", "Umidade", "Vento", "Datetime", "Temperatura"]

In [38]:
tempo["Vento"] = tempo["Vento"].str.replace(",", ".").astype(float)
tempo["Precipitacao"] = tempo["Precipitacao"].str.replace(",", ".").astype(float)

In [39]:
tempo['Descricao Chuva'] = tempo['Precipitacao'].apply(lambda x: 1 if (x > 0) & (x <= 2.5) else \
2 if (x > 2.5) & (x <= 10) else \
3 if (x > 10) else 0)

In [40]:
tempo['Calor'] = tempo['Temperatura'].apply(lambda x: '1' if (x > 16) & (x <= 25) else \
'2' if (x > 25) else '0')

In [41]:
tempo.dtypes

Precipitacao              float64
Umidade                     int64
Vento                     float64
Datetime           datetime64[ns]
Temperatura               float64
Descricao Chuva             int64
Calor                      object
dtype: object

In [42]:
tempo

,Precipitacao,Umidade,Vento,Datetime,Temperatura,Descricao Chuva,Calor
3624,0.0,51,2.5,2019-05-31 21:00:00,24.20,0,1
3625,3.8,85,4.5,2019-05-31 22:00:00,21.30,2,1
3626,3.0,89,0.3,2019-05-31 23:00:00,18.45,2,1
3627,0.0,88,0.0,2019-06-01 00:00:00,18.70,0,1
3628,0.0,86,0.4,2019-06-01 01:00:00,18.80,0,1
...,...,...,...,...,...,...,...
6787,0.0,59,3.8,2019-10-10 16:00:00,26.95,0,2
6788,0.0,70,3.4,2019-10-10 17:00:00,23.55,0,1
6789,0.0,77,1.8,2019-10-10 18:00:00,21.25,0,1
6790,0.0,81,2.0,2019-10-10 19:00:00,20.00,0,1


In [43]:
tempo.to_json("../Bases/Cleaned/weather.json")

In [44]:
df33642 = pd.read_csv("../Bases/Cleaned/rota33642.csv")
df32893 = pd.read_csv("../Bases/Cleaned/rota32893.csv")

In [45]:
df33642["data_partida2"] = pd.to_datetime(df33642["data_partida"])
df33642["data_partida2"] = df33642["data_partida2"].dt.round("H")

df32893["data_partida2"] = pd.to_datetime(df32893["data_partida"])
df32893["data_partida2"] = df32893["data_partida2"].dt.round("H")

In [46]:
df33642_merged = df33642.merge(tempo, left_on="data_partida2", right_on="Datetime", how="left")
df32893_merged = df32893.merge(tempo, left_on="data_partida2", right_on="Datetime", how="left")

In [47]:
df33642_merged.drop(['data_partida2','Datetime'],axis=1,inplace=True)
df32893_merged.drop(['data_partida2','Datetime'],axis=1,inplace=True)

In [48]:
df33642_merged.to_csv("../Bases/Cleaned/rota33642_weather.csv",index=False)
df32893_merged.to_csv("../Bases/Cleaned/rota32893_weather.csv",index=False)